In [ ]:
!pip install json
!pip install sklearn
!pip install sklearn.feature
!pip install spacy
!pip install nltk
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install pyldavis

In [ ]:
#imports go here, not all ended up being used etc, these are ones we used, some were tried and didnt work how expected or werent needed etc
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from nltk.corpus import stopwords
import spacy
from spacy.lang.en.examples import sentences
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string
import gensim
import os
import pickle
import pyLDAvis
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import time

## Read in datasets

In [ ]:
#read in review file.
data_file = open("yelp_academic_dataset_review.json", encoding="utf8")
data = []
for line in data_file:
    data.append(json.loads(line))
review_df = pd.DataFrame(data)
data_file.close()
#read in business file.
data_file = open("yelp_academic_dataset_business.json", encoding="utf8")
data = []
for line in data_file:
    data.append(json.loads(line))
business_df = pd.DataFrame(data)
data_file.close()

## another way to read in json file for review
with open('yelp_academic_dataset_business.json') as json_file:
    data = json_file.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects.
    data = list(map(json.loads, data))


## Exploratory Data Analyis

In [ ]:
import seaborn as sns
%matplotlib inline
plt.style.use('fivethirtyeight')

In [ ]:
business_df.head()
business_df.isnull().sum()

In [ ]:
plt.figure(figsize=(12,10))
f = sns.heatmap(business_df.isnull(),yticklabels=False, cbar=False, cmap = 'viridis')  # plotting the null values across all fields

In [ ]:
plt.figure(figsize=(6,6))
sns.countplot(x='is_open',data=business_df); #plotting the number of closed and open businesses

In [ ]:
sns.countplot(x='stars',data=business_df); # plotting the frequency of reviews across starts

In [ ]:
sns.distplot(business_df['review_count'].apply(np.log1p)); #plotting the review count

In [ ]:
by_state = business_df.groupby('state')   #grouping the data by state
by_state

In [ ]:
pip install squarify 

In [ ]:
import squarify    # pip install squarify (algorithm for treemap)
plt.figure(figsize=(12,12))

a = by_state['business_id'].count()   # identify the number of business ids by state

a.sort_values(ascending=False,inplace=True)

squarify.plot(sizes= a[0:15].values, label= a[0:15].index, alpha=0.9)

plt.axis('off')
plt.tight_layout()

In [ ]:
# plotting the number of businesses across different categories
cats=business_df['categories']  
cats
cats_ser = cats.value_counts()
cats_ser


cats_df = pd.DataFrame(cats_ser)
cats_df.reset_index(inplace=True)

plt.figure(figsize=(12,10))
f = sns.barplot( y= 'index',x = 'categories' , data = cats_df.iloc[0:20])
f.set_ylabel('Category')
f.set_xlabel('Number of businesses')

In [ ]:
#plotting word cloud for business names
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))

wordcloud = WordCloud(background_color='white',
                          width=1200,
                      stopwords = STOPWORDS,
                          height=1000
                         ).generate(str(business_df['name']))


plt.imshow(wordcloud)
plt.axis('off');

## data cleaning


In [ ]:
review_df = review_df.drop(['review_id','useful','funny','cool'], axis = 1)
# clean business file
business_df = pd.read_json(business_json, lines=True)

business_df_CA = df_business[df_business["state"]=="CA"]


#converting city to lower case
business_df_CA["city"] = business_df_CA.city.str.lower().apply(lambda x: x)
#removing spaces from the city
business_df_CA["city"] = business_df_CA.city.str.strip().apply(lambda x: x)
# removing extra characters from "santa barbara"
business_df_CA["city"] = business_df_CA.city.replace(to_replace="santa barbra", value="santa barbara")
business_df_CA["city"] = business_df_CA.city.replace(to_replace="santa barbara ap", value="santa barbara")
business_df_CA["city"] = business_df_CA.city.replace(to_replace="santa barbara,", value="santa barbara")
business_df_CA["city"] = business_df_CA.city.replace(to_replace="santa  barbara", value="santa barbara")
business_df_CA["city"] = business_df_CA.city.replace(to_replace="santa barbara & ventura counties", value="santa barbara")

#couting the occurances of cities
business_df_CA.groupby("city").city.count().sort_values(ascending = False)
#filtering out top 10 cities of california
business_df_CA = business_df_CA[(df_business_CA["city"] == "santa barbara") | (business_df_CA["city"] == "goleta")|
               (business_df_CA["city"] == "carpinteria")| (business_df_CA["city"] == "isla vista")|
               (business_df_CA["city"] == "montecito") | (business_df_CA["city"] == "summerland")

               ]
#writing the cleaned business file into .csv
business_df_CA.to_csv("C:/Users/raval/Desktop/Yelp Dataset/Business_Filtered.csv")
#reading business file into dataframe
business_df = pd.read_csv(business_df_CA)

In [ ]:
#merge business and review df
yelp_reviews = pd.merge(business_df, review_df, on="business_id", how="left")
# filtering out records with categories containing "restaurant" or "food"
yelp_reviews["categories"] = yelp.categories.str.lower().apply(lambda x: x)
yelp_reviews["boolean_findings_food"] = yelp.categories.str.contains("restaurant") | yelp.categories.str.contains("food")

#filter out food data
yelp_food = yelp[yelp["boolean_findings_food"]==True]
#filter out non food data
yelp_not_food = yelp[yelp["boolean_findings_food"]==False]
#store respective data into .csv files
yelp_not_food.to_csv("C:/Users/raval/Desktop/Yelp Dataset/yelp_not_food.csv")
yelp_food.to_csv("C:/Users/raval/Desktop/Yelp Dataset/yelp_food.csv")

In [ ]:
#these columns are dropped from the restaurant/food dataset
yelp_reviews = yelp_reviews.drop(['Unnamed: 0','Unnamed: 0.1','latitude','longitude',
                                  'hours', 'user_id', 'boolean_findings_food', 'address', 'postal_code',
                                  'business_id', 'state'], axis = 1)
#rename the columns for stars_x(business review rating) and stars y(actual review rating)
yelp_reviews.columns = ['business_rating' if x=='stars_x' else 'rating'
                        if x=='stars_y' else x for x in yelp_reviews.columns]


In [ ]:
#look at data types of our dataset
#we see that date is an object which may affect EDA later
# we also want to drop the time as it is unnecessary from any analysis
print(yelp_reviews.dtypes)

In [ ]:
#convert date to type datetime and drop the time
yelp_reviews['date'] =  pd.to_datetime(yelp_reviews['date'], infer_datetime_format=True)
yelp_reviews['date'] = yelp_reviews['date'].apply(lambda x: x.date())

In [ ]:
yelp_reviews.columns = ['business_rating' if x=='stars_x'
                        else 'rating' if x=='stars_y' else x for x in yelp_reviews.columns]

In [ ]:
yelp_reviews = yelp_reviews.drop(['Unnamed: 0','Unnamed: 0.1','latitude','longitude', 'hours', 'user_id', 'boolean_findings_food', 'address', 'postal_code', 'business_id', 'state'], axis = 1)

## here is the final df that is used for everything
##### can use this import here to bring it back in whenever

In [ ]:
yelp_reviews = pd.read_csv('/content/drive/Shareddrives/programming project/yelp_reviews_6A_v4.csv')

## text cleaning

In [ ]:
#stop words used for text processing
#loading the english language small model of spacy
en = spacy.load('en_core_web_sm')
stop_words = en.Defaults.stop_words
stop_words|= {'come','try','go','get','make','drink','restaurant','place',
                  'would','really','like','great','came','got', 'santa', 'barbara', 'order', 'plate', 'dish', 've', 'd', 's', 'll', 'm', 're'}
print(stop_words)

In [ ]:
#pre process the text analysis
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def lemmatization(texts):
     texts_out = []
     for sent in texts:
         doc = nlp(" ".join(sent))
         texts_out.append([token.lemma_ for token in doc])
     return texts_out


In [ ]:
yelp_reviews['new_text'] = strip_newline(yelp_reviews.text)
words = list(sent_to_words(yelp_reviews.new_text))
words = remove_stopwords(words)
bigram = bigrams(words)
bigram = [bigram[review] for review in words]
bigram = lemmatization(bigram)
yelp_reviews['new_text'] = bigram

In [ ]:
yelp_reviews['new_text'] = yelp_reviews['new_text'].apply(', '.join)

##sentiment analysis using vader

In [ ]:
#sentiment analysis add column giving polarization rating
sid = SentimentIntensityAnalyzer()
yelp_reviews['scores'] = yelp_reviews['new_text'].apply(lambda new_text: sid.polarity_scores(new_text))

In [ ]:
#get a compound score based off our sentiment scores of each word
yelp_reviews['polarity_score']  = yelp_reviews['scores'].apply(lambda score_dict: score_dict['compound'])
yelp_reviews.head()

In [ ]:
yelp_reviews.head()
#create column classifying as positive or negative
yelp_reviews['polarity'] = yelp_reviews['polarity_score'].apply(lambda c: 2 if c >= .3 else 0 if c <= -.3 else 1)
#convert to int
yelp_reviews['rating'] = yelp_reviews['rating'].astype(int)

In [ ]:
##use this if you want to only show final polarity score only
#yelp_reviews = yelp_reviews[['name', 'rating', 'new_text', 'polarity_score']]


In [ ]:
#positive wordcloud
 from PIL import Image
 from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


pos_review = yelp_reviews[yelp_reviews["polarity"]=="positive"]
txt = " ".join(review.lower() for review in pos_review["new_text"])
mask = np.array(Image.open("thumbsup.png"))
wordcloud = WordCloud(mask = mask, background_color='white', colormap='summer').generate(txt)
plt.imshow(wordcloud,interpolation = "bilinear")
plt.axis("off")
plt.savefig('positive_wc.png', bbox_inches='tight')
plt.show

In [ ]:
#negative word cloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

neg_review = yelp_reviews[yelp_reviews["polarity"] == "negative"]
txt = " ".join(review.lower() for review in neg_review["new_text"])
mask = np.array(Image.open("thumbsdown.png"))
wordcloud = WordCloud(mask=mask, background_color='white', colormap='autumn').generate(txt)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('negative_wc.png', bbox_inches='tight')
plt.show

## here is tfidf models

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
yelp_reviews['rating'] = yelp_reviews["rating"]-1
#ratings for labels werent matching up so had to do a rating scale of 0-4

##### below commented out cell can be used for binning and rerunning the tfid

In [ ]:
# yelp_reviews['rating'] = yelp_reviews['rating'].apply(lambda i: 2 if i >= 3 else 0 if i <= 1  else 1)

In [ ]:
# First, create a vectorizer object. we only consider 4000 words
x = yelp_reviews["new_text"].values.astype(str)
y = yelp_reviews["rating"].values


vectorizer = TfidfVectorizer(max_features=2000)


start = time.time()

x = vectorizer.fit_transform(x).toarray()

process_time = round(time.time()-start,2)

print("Vectorizing using TF-IDF approach took {} seconds".format(process_time))

x.shape

In [ ]:
## sometimes this was needed to label encode target variable not always tho
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

## xgboost

In [ ]:
from xgboost import XGBClassifier
start = time.time()

evalset = [(x_train, y_train), (x_test,y_test)]
classifier = XGBClassifier(n_estimators=100, learning_rate=.01, eval_metric='mlogloss', num_class = 1)

classifier.fit(x_train,y_train, eval_set=evalset)

preds = classifier.predict(x_test)

process_time = round(time.time()-start,2)

In [ ]:
print("XGB classification took {} seconds".format(process_time))
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, preds)
print("Accuracy of our XGB Classifier is: %.2f%%" % (accuracy * 100.0))
#show log loss learning curve
from matplotlib import pyplot
# plot learning curves
pyplot.plot(results['validation_0']['mlogloss'], label='train')
pyplot.plot(results['validation_1']['mlogloss'], label='test')
# show the legend
pyplot.legend()
pyplot.savefig('logloss_1_5')
# show the plot
pyplot.show()

In [ ]:
##plot our confusion matrix

pyplot.show()
plt.subplots(figsize=(6, 6))
sns.heatmap(confusion_matrix(y_pred=preds, y_true=y_test), annot=True, fmt=".1f", linewidths=1.5, cmap="autumn")
plt.savefig('boost_cm.png')
plt.show()

## naive bayes

In [ ]:
# First, create a vectorizer object. we only consider 4000 words
x = yelp_reviews["new_text"].values.astype(str)
y = yelp_reviews["rating"].values


vectorizer = TfidfVectorizer(max_features=2000)


start = time.time()

x = vectorizer.fit_transform(x).toarray()

process_time = round(time.time()-start,2)

print("Vectorizing using TF-IDF approach took {} seconds".format(process_time))

x.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train, y_train)
nb_preds = clf.predict(x_test)
nb_acc = accuracy_score(y_test, nb_preds)
print("Accuracy of our nb Classifier is: %.2f%%" % (nb_acc * 100.0))

## logistic regression

In [ ]:
# First, create a vectorizer object. we only consider 4000 words
x = yelp_reviews["new_text"].values.astype(str)
y = yelp_reviews["rating"].values


vectorizer = TfidfVectorizer(max_features=2000)


start = time.time()

x = vectorizer.fit_transform(x).toarray()

process_time = round(time.time()-start,2)

print("Vectorizing using TF-IDF approach took {} seconds".format(process_time))

x.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
clf = LogisticRegression(max_iter=10000).fit(x_train, y_train)
log_preds = clf.predict(x_test)
log_acc = accuracy_score(y_test, log_preds)

In [ ]:
print("Accuracy of our log Classifier is: %.2f%%" % (log_acc * 100.0))

## topic modeling

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [ ]:
#def for text processing again if needed
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

def lemmatization(texts):
     texts_out = []
     for sent in texts:
         doc = nlp(" ".join(sent))
         texts_out.append([token.lemma_ for token in doc])
     return texts_out

In [ ]:
def get_corpus(df):
#this gets all of our pieces needed for lda

    df['new_text'] = strip_newline(df.new_text)
    words = list(sent_to_words(df.new_text))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    lemma = lemmatization(bigram)
    id2word = gensim.corpora.Dictionary(lemma)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in lemma]
    return corpus, id2word, bigram, lemma

In [ ]:
#use get_corp function to get our corpus for lda
corpus, id2word, bigram, lemma = get_corpus(yelp_reviews)

In [ ]:
#this is lda model, use multicore to use more cpu processing power
lda_model = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus,
    num_topics=20,
    id2word=id2word,
    chunksize=100,
    workers=11, # Change to Num. Processing Cores - 1
    passes=50,
    eval_every = 1,
    per_word_topics=True)
lda_model.save('lda_model.model')

In [ ]:
#print topics
lda_model.print_topics(20,num_words=20)[:20]

In [ ]:
#create interactive graphic and save it for use on github
pyLDAvis.enable_notebook()
viz = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

pyLDAvis.save_html(viz, 'lda_V2.html')

In [ ]:
#map the reviews data to the model, v is the reviews transformed to word vectors by doc2bow
yelp_map = lda_model[corpus[0:len(corpus)]]

#assign topic to each review
import operator
topic = []
for i in yelp_map:
    #find the topic with the highest proportions
    t = max(i, key = operator.itemgetter(1))
    topic.append(t[0])

In [ ]:
#change topics to actual words/categories
topic_name = {0:'Atmosphere', 1:'Food', 2:'Atmosphere',
             3:'Food', 4:'Service', 5:'Food',
             6:'Price', 7:'Service', 8:'Food', 9:'Atmosphere', 10:'Food', 11:'Service', 12:'Food', 13:'Food', 14:'Atmosphere', 15:'Food', 16:'Food', 17:'Atmosphere', 18:'Service', 19:'Price'}

yelp_reviews = yelp_reviews.replace({'topic':topic_name})

In [ ]:
#sentiment score again, this is here because when we originally did this we did
#not have it added to that dataset 
analyzer = SentimentIntensityAnalyzer()
# Vader sentiment analysis

def sentiment_analyzer_scores(review, nth):
    score_list = []
    for i in range(nth):
        score = analyzer.polarity_scores(review[i])
        for cat, point in score.items():
            #only take compound score
            if cat == 'compound':
                score_list.append(point)
    return score_list

In [ ]:
#create polarity score 
def topic_scores(dataset):

    reviews = dataset['new_text']
    reviews_score = sentiment_analyzer_scores(reviews, len(reviews))
    reviews_score = pd.DataFrame(reviews_score)
    yelp_reviews['topic_score'] = reviews_score
    business_scores = yelp_reviews.loc[:, ['name', 'topic', 'topic_score']]

    #normalize the score to 0-5
    business_scores = round(abs((business_scores.groupby(['name', 'topic']).sum() /
                   business_scores.groupby(['name', 'topic']).count()) * 5), 1)

    pd.set_option('display.max_rows', 100)
    return business_scores

categorized_score= topic_scores(yelp_reviews)
categorized_score

In [ ]:
#apply polarity score from our dataset with this version here
# def topic_scores(dataset):

#     reviews = dataset['new_text']
#     business_scores = yelp_reviews.loc[:, ['name', 'topic', 'polarity_score']]

#     #normalize the score to 0-5
#     business_scores = round(abs((business_scores.groupby(['name', 'topic']).sum() /
#                    business_scores.groupby(['name', 'topic']).count()) * 5), 1)

#     pd.set_option('display.max_rows', 100)
#     return business_scores

# categorized_score= topic_scores(yelp_reviews)
# categorized_score